# **<h1><center> Production EDA </center></h1>**

## Import Packages

In [73]:
import pandas as pd
import sqlite3
import plotly.io as pio
import numpy as np
from statistics import mean 
import plotly.express as px
import plotly.graph_objects as go
pd.set_option("display.max_rows",1000)
pd.options.display.precision = 1
%load_ext sql
%sql sqlite:///C:\Users\johno\Python\Scrapy\NDIC\Well_DataBase.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Import Production data

In [10]:
query = %sql SELECT * FROM prod_table_clean WHERE Days > 0

df_production = query.DataFrame()
df_production.set_index('UWI',inplace=True)
df_production['Date'] = pd.to_datetime(df_production['Date'], infer_datetime_format=True)
df_production.describe().transpose()

 * sqlite:///C:\Users\johno\Python\Scrapy\NDIC\Well_DataBase.db
Done.


,count,mean,std,min,25%,50%,75%,max
Days,1.1e+06,27.5,6.5,1.0,28.0,30.0,31.0,3.1e+01
Oil,1.1e+06,3251.9,4741.6,0.0,896.0,1737.0,3500.0,1.4e+05
Water,1.1e+06,2979.5,5544.6,0.0,613.0,1407.0,3040.0,6.5e+05
Gas,1.1e+06,5191.4,8670.4,0.0,1114.0,2469.0,5228.0,2.0e+05
Fluid,1.1e+06,6231.4,9342.1,0.0,1824.0,3363.0,6588.0,6.5e+05
WOR,1.1e+06,inf,NaN,0.0,0.4,0.8,1.5,inf
GOR,1.1e+06,inf,NaN,0.0,0.9,1.3,2.2,inf


## Import Header data

In [64]:
query = %sql SELECT * FROM header_table_clean

df_header = query.DataFrame()

# Set Index and view shape
df_header.set_index('UWI',inplace=True)
df_header[['Completion_Date','Spud_Date']] = df_header[['Completion_Date','Spud_Date']].apply(pd.to_datetime, errors='coerce')
df_header.sample(5)

 * sqlite:///C:\Users\johno\Python\Scrapy\NDIC\Well_DataBase.db
Done.


,File_Number,Well_Name,Well_Type,Well_Status,Location,Latitude,Longitude,Current_Operator,Completion_Date,Total_Depth,...,Bottom_Perf,Operator,desete,Perforated_Length,Direction,TRS,Section,Township,Range,Block
UWI,,,,,,,,,,,,,,,,,,,,,
33105029360000,24604,P WOOD 154-98-4-27-34-13HB,OG,A,NWNW 27-154-98,48.1,-103.3,WHITING OIL AND GAS CORPORATION,2013-07-29,20800,...,20670.0,WHITING,OIL AND GAS CORPORATION,8979.0,NWNW,27-154-98,27,154,98,15498
33053046730000,24621,BOBBY LEE USA 41-30H,OG,A,NENE 30-152-93,48.0,-102.6,MARATHON OIL COMPANY,2018-12-29,20858,...,20720.0,MARATHON,OIL COMPANY,9667.0,NENE,30-152-93,30,152,93,15293
33105042290000,32340,CARSCALLEN 31-14-4H,OG,A,NENW 14-154-98,48.2,-103.2,WHITING OIL AND GAS CORPORATION,2016-06-14,23250,...,23043.0,WHITING,OIL AND GAS CORPORATION,11670.0,NENW,14-154-98,14,154,98,15498
33053055950000,27413,TARPON FEDERAL 44-19TFHU,OG,IA,SESE 19-153-96,48.1,-103.1,WHITING OIL AND GAS CORPORATION,2015-02-17,20420,...,19421.0,WHITING,OIL AND GAS CORPORATION,8395.0,SESE,19-153-96,19,153,96,15396
33053048780000,25229,KOESER 41X-15H,OG,A,NENE 15-150-97,47.8,-103.1,XTO ENERGY INC,2013-10-07,20632,...,20632.0,XTO,ENERGY INC,9418.0,NENE,15-150-97,15,150,97,15097


## Define Functions

In [78]:
def STREAM_PLOT(dataframe, production_dataframe, material, width, height, variable_dict = {'0-99999999':None}, rate_cum = 0, averages = 0, cumulative = 1,
                line_width = 1, variable = df_header.iloc[:,0].name, all_streams = 1):
    
    fig = go.Figure(layout={'height':height,'width':width})
    
    if averages == 1:
        fig.update_layout(xaxis_title="Producing Days")
        for item in variable_dict.keys():
            low = float(item.split('-')[0])
            high = float(item.split('-')[1]) 
            df_temp = dataframe[dataframe[variable].between(low,high)]
            if len(df_temp) >= 1:       
                
                if material in ['WOR','GOR']:
                    average_list = []
                    for m in [i for i in range(1,27,1)]:
                        prod_list = []
                        for uwi in df_temp.index.tolist():  
                            try:
                                cum_day_list = production_dataframe.loc[uwi,'Days'].cumsum().tolist()
                                if m * 30.4 < max(cum_day_list):                                
                                    if cumulative == 1:
                                        cum_prod_list = production_dataframe.loc[uwi,material].expanding().mean().tolist()
                                        prod_list.append(np.interp( m*30.4 , cum_day_list , cum_prod_list))
                                        name = f"Cumulative {material}"
                                    else:                 
                                        rate_prod_list = production_dataframe.loc[uwi,material].tolist()
                                        prod_list.append(np.interp( m*30.4 , cum_day_list , rate_prod_list)) 
                                        name = f"Monthly {material}"
                            except: None
                        try:
                            average_list.append(mean(prod_list))
                        except: 
                            None
                    fig.update_layout(yaxis_title=name)
                    fig.add_trace(go.Scatter(x=[i*30.4 for i in range(1,27,1)], y=average_list, mode='lines',name=f'{item} Average', line=dict(color=variable_dict[item], width=6)))

                elif rate_cum == 0:        
                    average_list = []
                    for m in [i for i in range(1,27,1)]:
                        prod_list = []
                        for uwi in df_temp.index.tolist():   
                            cum_day_list = production_dataframe.loc[uwi,'Days'].cumsum().tolist()
                            if m * 30.4 < max(cum_day_list):
                                if cumulative == 1:
                                    cum_prod_list = production_dataframe.loc[uwi,material].cumsum().tolist()
                                    prod_list.append(np.interp( m*30.4 , cum_day_list , cum_prod_list))
                                    name = f"Cumulative {material} bbl"
                                else:                 
                                    rate_prod_list = production_dataframe.loc[uwi,material].tolist()
                                    prod_list.append(np.interp( m*30.4 , cum_day_list , rate_prod_list)) 
                                    name = f"Monthly {material} bbl"
                        try:
                            average_list.append(mean(prod_list))
                        except: 
                            None
                    fig.update_layout(yaxis_title =name)
                    fig.add_trace(go.Scatter(x=[i*30.4 for i in range(1,27,1)], y=average_list, mode='lines',name=f'{item} Average', line=dict(color=variable_dict[item], width=6)))
            print(item,f'{len(df_temp)} wells')
            
    if all_streams == 1:
        for item in variable_dict.keys():
            low = float(item.split('-')[0])
            high = float(item.split('-')[1]) 
            df_temp = dataframe[dataframe[variable].between(low,high)]
            if len(df_temp) >= 1:

                if material in ['WOR','GOR']:
                    for uwi in df_temp.index.tolist():
                        try:
                            day_list = production_dataframe.loc[uwi,'Days'].cumsum().tolist()
                            if cumulative == 1:
                                prod_list = production_dataframe.loc[uwi,material].expanding().mean().tolist()
                                name = f"Cumulative {material}"
                            else:
                                prod_list = production_dataframe.loc[uwi,material].tolist()
                                name = f"Monthy {material} Rate"
                            fig.add_trace(go.Scatter(x=day_list, y=prod_list, mode='lines',name=str(dataframe.loc[uwi,'Well_Name']), line=dict(width=line_width,color=variable_dict[item])))
                        except: None
                    fig.update_layout(xaxis_title="Producing Days", yaxis_title=name)

                elif rate_cum == 1:        
                    for uwi in df_temp.index.tolist():
                        rate_list = production_dataframe.loc[uwi,material].tolist()
                        cum_list = production_dataframe.loc[uwi,material].cumsum().tolist()
                        fig.update_yaxes(type='log')  
                        fig.add_trace(go.Scatter(x=cum_list, y=rate_list, mode='lines',name=str(dataframe.loc[uwi,'Well_Name']), line=dict(width=line_width,color=variable_dict[item])))
                        fig.update_layout(xaxis_title= material+' Cum', yaxis_title =material+' Rate')

                elif rate_cum == 0:       
                    fig.update_layout(xaxis_title="Producing Days")
                    for uwi in df_temp.index.tolist():
                        day_list = production_dataframe.loc[uwi,'Days'].cumsum().tolist()
                        if cumulative == 1:
                            prod_list = production_dataframe.loc[uwi,material].cumsum().tolist()
                            name = f"Cumulative {material} bbl"
                        else:
                            prod_list = production_dataframe.loc[uwi,material].tolist()
                            name = f"Monthy {material} bbl"
                            fig.update_yaxes(type='log')  
                        fig.add_trace(go.Scatter(x=day_list, y=prod_list, mode='lines',name=str(dataframe.loc[uwi,'Well_Name']), line=dict(width=line_width,color=variable_dict[item])))
                        fig.update_layout(yaxis_title=name)

    return fig                         

## What does the basin look like?

In [ ]:
df_sample = df_header[df_header['Top_Perf'].between(9000,11500)]

fig = go.Figure(data=go.Scattergeo(
    lon = df_sample['Longitude'],
    lat = df_sample['Latitude'],
    text = df_sample[['Well_Name','Top_Perf','Current_Operator']],
    marker = dict(
        color = df_sample['Top_Perf'],
        colorscale = 'rainbow',
        reversescale = True,
        opacity = 0.7,
        size = 7,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 0.1
        )
    )
))

fig.update_layout(
        title = 'Williston Basin Wells',
        geo_scope='usa',width = 1200, height = 800,
    geo = dict()
    )

fig.update_geos(
    center=dict(lon=df_sample['Longitude'].mean(), lat=df_sample['Latitude'].mean()),
    lataxis_range=[df_sample['Latitude'].min(),df_sample['Latitude'].max()], lonaxis_range=[df_sample['Longitude'].min(),df_sample['Longitude'].max()]
               )
  
fig.show()

## What Operator has Produced the Most Oil to Date?

In [ ]:
%%sql 

SELECT p.UWI, COUNT(DISTINCT p.UWI) AS 'Wells', SUM(p.Oil) AS 'Cumulative_Oil', h.Current_Operator
FROM prod_table p 
JOIN header_table h 
ON p.UWI = h.UWI 
GROUP BY Current_Operator
ORDER BY Cumulative_Oil desc
LIMIT 8

##### Plot

In [ ]:
query = %sql SELECT p.UWI, COUNT(DISTINCT p.UWI) AS 'Wells', SUM(p.Oil) AS 'Cumulative_Oil', h.Current_Operator FROM prod_table p JOIN header_table h ON p.UWI = h.UWI GROUP BY Current_Operator ORDER BY Cumulative_Oil desc LIMIT 8
df = query.DataFrame()

fig = px.bar(df, x='Current_Operator', y='Cumulative_Oil', width = 700, height=500)
fig.show()

## What Operator has Produced the Most Oil for their well count?

In [ ]:
%%sql 

SELECT p.UWI, COUNT(DISTINCT p.UWI) AS 'Wells', SUM(p.Oil) AS 'Cumulative_Oil', h.Current_Operator
FROM prod_table p 
JOIN header_table h 
ON p.UWI = h.UWI 
GROUP BY Current_Operator
ORDER BY Cumulative_Oil desc
LIMIT 8

#### Plot

In [ ]:
query = %sql SELECT p.UWI, COUNT(DISTINCT p.UWI) AS 'Wells', SUM(p.Oil) AS 'Cumulative_Oil', h.Current_Operator FROM prod_table p JOIN header_table h ON p.UWI = h.UWI GROUP BY Current_Operator ORDER BY Cumulative_Oil desc LIMIT 10
df = query.DataFrame()
df['Oil_Per_Well'] = df['Cumulative_Oil'] / df['Wells']
df.sort_values('Oil_Per_Well',inplace=True,ascending=False)

fig = px.bar(df, x='Current_Operator', y='Oil_Per_Well', width = 700, height=500)
fig.show()

## Simple Production Plot

In [ ]:
df9 = df_header[df_header['Current_Operator'].str.contains('XTO')].sample(8)
df_prod = df_production.copy()
# ---------------------------------------------------------
STREAM_PLOT(dataframe = df9, 
            production_dataframe = df_prod, 
            material = 'Oil', 
            cumulative = 1, 
            line_width = 2,
            width = 700, height = 500 )

## Water / Oil Ratio Plot

In [ ]:
df9 = df_header[df_header['Vintage_Year'] > 2008].sample(1500)
df_prod = df_production[df_production['WOR'].between(0,10)]
# ---------------------------------------------------------
STREAM_PLOT(dataframe = df9, 
            production_dataframe = df_prod, 
            material = 'WOR', 
            cumulative = 0, 
            variable = 'Vintage_Year', 
            variable_dict = {'2008-2012':'grey','2012-2014':'gold','2014-2016':'orange','2016-2018':'red','2018-2020':'blue'},
            averages = 1,
            all_streams = 0,
            width = 800, height = 600 )

## Production by ~Depth / Pressure

In [ ]:
df9 = df_header.sample(5000)
df_prod = df_production.copy()
# ---------------------------------------------------------
STREAM_PLOT(dataframe = df9, 
            production_dataframe = df_prod, 
            material = 'Oil', 
            cumulative =10, 
            variable = 'Top_Perf', 
            variable_dict = {'0-9000':'grey','9000-1000':'gold','10000-11000':'orange','11000-12000':'red','12000-13000':'blue'},
            averages = 1,
            all_streams = 0,
            width = 800, height = 600 )

## What areas have produced the most oil?

In [65]:
query = %sql SELECT p.UWI, h.Block, COUNT(DISTINCT p.UWI) AS 'Wells_Per_Block', SUM(p.Oil) AS 'Cumulative_Oil_Per_Block' FROM prod_table_clean p JOIN header_table_clean h ON p.UWI = h.UWI GROUP BY Block ORDER BY Cumulative_Oil_Per_Block desc

df_block = query.DataFrame()
df_block.drop(columns='UWI',inplace=True)
df_block.set_index('Block',inplace=True)
df_block['Cum_Oil_Per_Well_Block'] = df_block['Cumulative_Oil_Per_Block'] / df_block['Wells_Per_Block']
print(len(df_block))
df_block.head()

 * sqlite:///C:\Users\johno\Python\Scrapy\NDIC\Well_DataBase.db
Done.
327


,Wells_Per_Block,Cumulative_Oil_Per_Block,Cum_Oil_Per_Well_Block
Block,,,
15095,212,68701132,324061.9
14995,177,59061427,333680.4
15294,148,53781107,363385.9
15093,134,52422937,391215.9
14796,156,52219525,334740.5


In [66]:
df_header = df_header.merge(df_block[['Wells_Per_Block','Cumulative_Oil_Per_Block','Cum_Oil_Per_Well_Block']],left_on='Block',right_on='Block')
df_header.sort_values('Cumulative_Oil_Per_Block',inplace=True,ascending=False)
df_header.head()

,File_Number,Well_Name,Well_Type,Well_Status,Location,Latitude,Longitude,Current_Operator,Completion_Date,Total_Depth,...,Perforated_Length,Direction,TRS,Section,Township,Range,Block,Wells_Per_Block,Cumulative_Oil_Per_Block,Cum_Oil_Per_Well_Block
3627,20780,HENDERSON 1-28/33H,OG,A,NENE 28-150-95,47.8,-102.8,QEP ENERGY COMPANY,2012-01-17,20682,...,9452.0,NENE,28-150-95,28,150,95,15095,212,68701132,324061.9
3744,30583,TIPI V 1-24-25TH,OG,A,SESE 13-150-95,47.8,-102.8,QEP ENERGY COMPANY,2018-04-06,21349,...,10202.0,SESE,13-150-95,13,150,95,15095,212,68701132,324061.9
3731,30048,MARLENE 42X-20F2,OG,A,SENE 20-150-95,47.8,-102.9,XTO ENERGY INC,2015-07-25,20917,...,9780.0,SENE,20-150-95,20,150,95,15095,212,68701132,324061.9
3732,30049,MARLENE 42X-20C,OG,A,SENE 20-150-95,47.8,-102.9,XTO ENERGY INC,2015-08-04,20608,...,9651.0,SENE,20-150-95,20,150,95,15095,212,68701132,324061.9
3733,30214,MARLENE 42X-20CXD,OG,A,SENE 20-150-95,47.8,-102.9,XTO ENERGY INC,2015-07-07,20567,...,9583.0,SENE,20-150-95,20,150,95,15095,212,68701132,324061.9


#### Map

In [ ]:
df_sample = df_header[df_header['Top_Perf'].between(9000,11500)]

fig = go.Figure(data=go.Scattergeo(
    lon = df_sample['Longitude'],
    lat = df_sample['Latitude'],
    text = df_sample[['Well_Name','Top_Perf','Current_Operator','Block','Cumulative_Oil_Per_Block']],
    marker = dict(
        color = df_sample['Cumulative_Oil_Per_Block'],
        colorscale = 'rainbow',
        reversescale = True,
        opacity = 0.7,
        size = 7,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 0.1
        )
    )
))

fig.update_layout(
        title = 'Williston Basin Wells',
        geo_scope='usa',width = 1200, height = 800,
    geo = dict()
    )

fig.update_geos(
    center=dict(lon=df_sample['Longitude'].mean(), lat=df_sample['Latitude'].mean()),
    lataxis_range=[df_sample['Latitude'].min(),df_sample['Latitude'].max()], lonaxis_range=[df_sample['Longitude'].min(),df_sample['Longitude'].max()]
               )
  
fig.show()

## What areas have produced the most oil per well?

### Map

In [ ]:
df_sample = df_header[df_header['Top_Perf'].between(9000,11500)]

fig = go.Figure(data=go.Scattergeo(
    lon = df_sample['Longitude'],
    lat = df_sample['Latitude'],
    text = df_sample[['Well_Name','Top_Perf','Current_Operator','Block','Cumulative_Oil_Per_Block','Cum_Oil_Per_Well_Block']],
    marker = dict(
        color = df_sample['Cum_Oil_Per_Well_Block'],
        colorscale = 'rainbow',
        reversescale = True,
        opacity = 0.7,
        size = 7,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 0.1
        )
    )
))

fig.update_layout(
        title = 'Williston Basin Wells',
        geo_scope='usa',width = 1200, height = 800,
    geo = dict()
    )

fig.update_geos(
    center=dict(lon=df_sample['Longitude'].mean(), lat=df_sample['Latitude'].mean()),
    lataxis_range=[df_sample['Latitude'].min(),df_sample['Latitude'].max()], lonaxis_range=[df_sample['Longitude'].min(),df_sample['Longitude'].max()]
               )
  
fig.show()